In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

In [4]:
# Tentukan perangkat yang digunakan (GPU jika tersedia, jika tidak maka CPU)
device_cpu = torch.device("cpu")
device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device_cpu

device(type='cpu')

In [6]:
device_gpu

device(type='cuda')

In [25]:
import torch

# Pastikan CUDA tersedia
if torch.cuda.is_available():
    # Mendapatkan jumlah GPU yang tersedia
    num_gpus = torch.cuda.device_count()
    
    # Menampilkan informasi setiap GPU
    for i in range(num_gpus):
        print(f"--- GPU {i} ---")
        print(f"Nama GPU: {torch.cuda.get_device_name(i)}")
        print(f"Memori Total: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")
        print(f"Memori Alokasi: {torch.cuda.memory_allocated(i) / 1024**3:.2f} GB")
        print(f"Memori Tersedia: {torch.cuda.memory_reserved(i) / 1024**3:.2f} GB")
        print(f"MultiProcessor Count: {torch.cuda.get_device_properties(i).multi_processor_count}")
else:
    print("CUDA tidak tersedia, pastikan perangkat memiliki GPU yang mendukung CUDA.")


--- GPU 0 ---
Nama GPU: NVIDIA GeForce MX150
Memori Total: 2.00 GB
Memori Alokasi: 0.02 GB
Memori Tersedia: 0.02 GB
MultiProcessor Count: 3


In [8]:
# Definisikan transformasi untuk data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [9]:
# Load dataset MNIST
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:03<00:00, 2594925.45it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 120945.69it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 834058.41it/s] 


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2226049.17it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [10]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [12]:
# Definisikan model sederhana
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the image
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [16]:
# Fungsi untuk melatih model
def train_model(device):
    model = SimpleNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 1
    start_time = time.time()

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Print statistics
            running_loss += loss.item()
            if i % 100 == 99:  # print every 100 mini-batches
                print(f'Epoch [{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0

    total_time = time.time() - start_time
    print(f'Training time on {device}: {total_time:.2f} seconds')

In [24]:
# Latih model di CPU
print("Training on CPU:")
train_model(device_cpu)

Training on CPU:
Epoch [1, 100] loss: 1.002
Epoch [1, 200] loss: 0.421
Epoch [1, 300] loss: 0.361
Epoch [1, 400] loss: 0.319
Epoch [1, 500] loss: 0.322
Epoch [1, 600] loss: 0.291
Epoch [1, 700] loss: 0.273
Epoch [1, 800] loss: 0.258
Epoch [1, 900] loss: 0.239
Training time on cpu: 24.16 seconds


In [23]:
# Latih model di GPU (jika tersedia)
if torch.cuda.is_available():
    print("\nTraining on GPU:")
    train_model(device_gpu)
else:
    print("\nGPU is not available, skipping GPU training.")


Training on GPU:
Epoch [1, 100] loss: 1.038
Epoch [1, 200] loss: 0.437
Epoch [1, 300] loss: 0.374
Epoch [1, 400] loss: 0.345
Epoch [1, 500] loss: 0.334
Epoch [1, 600] loss: 0.305
Epoch [1, 700] loss: 0.291
Epoch [1, 800] loss: 0.253
Epoch [1, 900] loss: 0.259
Training time on cuda: 46.78 seconds
